**Model 4 Random Forest Churn Prediction**

# Importing Libraries

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [2]:
df = pd.read_csv("/kaggle/input/bank-data/bank_data_train.csv", nrows=10000)
df.head()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,146841,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
1,146842,0,0.041033,NaN,NaN,NaN,NaN,0.0,0.166667,0.186107,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
2,146843,0,0.006915,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
3,146844,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
4,146845,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0


In [3]:
df.describe()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,10000.000000,10000.000000,8884.000000,4335.0,1709.000000,4335.0,4335.0,10000.000000,2078.000000,2078.000000,...,10000.000000,2771.000000,2714.000000,2833.000000,2771.000000,10000.000000,2714.000000,2714.000000,2714.000000,10000.000000
mean,153144.826100,0.109000,0.043817,0.0,51.222937,0.0,0.0,0.001420,0.392976,0.394723,...,0.008282,0.051846,0.026016,0.051123,0.012450,0.004529,0.014107,0.014107,0.014107,0.076000
std,3625.391726,0.448708,0.110350,0.0,21.786972,0.0,0.0,0.029349,0.307262,0.329538,...,0.070735,0.210160,0.121445,0.191762,0.106335,0.061504,0.102023,0.102023,0.102023,0.265011
min,146841.000000,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.015873,0.000488,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,150007.750000,0.000000,0.000000,0.0,31.000000,0.0,0.0,0.000000,0.166667,0.131588,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,153137.500000,0.000000,0.000000,0.0,54.000000,0.0,0.0,0.000000,0.274836,0.270367,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,156296.250000,0.000000,0.035577,0.0,73.000000,0.0,0.0,0.000000,0.500000,0.602963,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,159411.000000,7.000000,1.000000,0.0,89.000000,0.0,0.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Pre-processing

In [5]:
#find unique values for categorical columns
for col in df.select_dtypes(include='object').columns:
  print(col)
  print(df[col].unique())
  print()

CLNT_TRUST_RELATION
[nan 'MOTHER' 'BROTHER' 'FRIEND' 'SISTER' 'OTHER' 'RELATIVE' 'DAUGHTER'
 'SON' 'FATHER' 'Близкий ро' 'Друг' 'Отец' 'Сестра' 'Сын' 'Мать' 'Муж']

APP_MARITAL_STATUS
[nan 'D' 'M' 'v' 'V' 'T' 'm' 'N' ' ' 'W' 't' 'd' 'C']

APP_KIND_OF_PROP_HABITATION
[nan 'SO' 'RENT' 'OTHER' 'JO' 'NPRIVAT']

CLNT_JOB_POSITION_TYPE
[nan 'SPECIALIST' 'SELF_EMPL' 'MANAGER' 'TOP_MANAGER']

CLNT_JOB_POSITION
['начальник отдела' nan 'ГЕНЕРАЛЬНЫЙ ДИРЕКТОР' ...
 'Директор съемочной группы' 'ПОВАР ГОРЯЧЕГО ЦЕХА'
 'директор по производству']

APP_DRIVING_LICENSE
[nan 'Y' 'N']

APP_EDUCATION
[nan 'S' 'HH' 'h' 'H' 'SS' 'UH' 'A' 's' 'i' 'US' 'e' 'I' 'HI' 'a']

APP_TRAVEL_PASS
[nan 'N' 'Y']

APP_CAR
[nan 'N' 'Y']

APP_POSITION_TYPE
[nan 'SPECIALIST' 'SELF_EMPL' 'MANAGER' 'TOP_MANAGER']

APP_EMP_TYPE
[nan 'PRIVATE' 'STATE' 'IP' 'INTER']

APP_COMP_TYPE
[nan 'PRIVATE' 'STATE' 'IP' 'INTER']

PACK
['K01' '102' '105' 'O01' '103' '101' '107' '301' '104' '108' '109']



In [6]:
mis_value = df.isnull().sum() / len(df) * 100
mis_value[mis_value > 0].sort_values(ascending=False) # find missing values in each column

CLNT_SALARY_VALUE          99.79
LDEAL_YQZ_COM              99.65
LDEAL_YQZ_CHRG             99.61
AVG_PCT_MONTH_TO_PCLOSE    99.51
LDEAL_AMT_MONTH            99.44
                           ...  
TRANS_COUNT_ATM_PRC        11.16
TRANS_COUNT_NAS_PRC        11.16
AMOUNT_RUB_ATM_PRC         11.16
TRANS_COUNT_SUP_PRC        11.16
AMOUNT_RUB_NAS_PRC         11.16
Length: 81, dtype: float64

In [7]:
df.drop(['PRC_ACCEPTS_A_EMAIL_LINK','PRC_ACCEPTS_A_POS','PRC_ACCEPTS_A_TK','PRC_ACCEPTS_A_AMOBILE','PRC_ACCEPTS_TK','PRC_ACCEPTS_A_MTP','CNT_ACCEPTS_TK','PRC_ACCEPTS_A_ATM','PRC_ACCEPTS_MTP','CNT_ACCEPTS_MTP'], axis=1, inplace=True)
df.drop(['APP_MARITAL_STATUS','CLNT_TRUST_RELATION','APP_CAR','APP_DRIVING_LICENSE','CLNT_JOB_POSITION','APP_TRAVEL_PASS','APP_COMP_TYPE'], axis=1, inplace=True)

In [8]:
cat_cols = df.select_dtypes(include=['object','category']).columns
num1_cols = df.select_dtypes(include=['float64']).columns
num2_cols = df.select_dtypes(include=['int64']).columns

print("Numerical Columns with float value:\n ",num1_cols)
print()
print("Numerical Columns with integer value:\n ",num2_cols)
print()
print("Categorical Columns:\n ",cat_cols) #seperate categorical & numerical columns

Numerical Columns with float value:
  Index(['AMOUNT_RUB_CLO_PRC', 'APP_REGISTR_RGN_CODE', 'TURNOVER_DYNAMIC_IL_1M',
       'CNT_TRAN_AUT_TENDENCY1M', 'SUM_TRAN_AUT_TENDENCY1M',
       'AMOUNT_RUB_SUP_PRC', 'SUM_TRAN_AUT_TENDENCY3M', 'REST_DYNAMIC_FDEP_1M',
       'CNT_TRAN_AUT_TENDENCY3M', 'REST_DYNAMIC_SAVE_3M', 'REST_AVG_CUR',
       'CNT_TRAN_MED_TENDENCY1M', 'AMOUNT_RUB_NAS_PRC', 'TRANS_COUNT_SUP_PRC',
       'CNT_TRAN_CLO_TENDENCY1M', 'SUM_TRAN_MED_TENDENCY1M',
       'TRANS_COUNT_NAS_PRC', 'SUM_TRAN_CLO_TENDENCY1M', 'TURNOVER_CC',
       'TRANS_COUNT_ATM_PRC', 'AMOUNT_RUB_ATM_PRC', 'TURNOVER_PAYM',
       'CNT_TRAN_MED_TENDENCY3M', 'SUM_TRAN_MED_TENDENCY3M',
       'REST_DYNAMIC_FDEP_3M', 'REST_DYNAMIC_IL_1M', 'SUM_TRAN_CLO_TENDENCY3M',
       'LDEAL_TENOR_MAX', 'LDEAL_YQZ_CHRG', 'DEAL_YQZ_IR_MAX', 'LDEAL_YQZ_COM',
       'DEAL_YQZ_IR_MIN', 'CNT_TRAN_CLO_TENDENCY3M', 'REST_DYNAMIC_CUR_1M',
       'REST_AVG_PAYM', 'LDEAL_TENOR_MIN', 'LDEAL_AMT_MONTH',
       'LDEAL_GRACE_DAYS_PCT

In [19]:
#filled categorical columns with the help of 'mode'
cat_cols_to_fill = ['APP_KIND_OF_PROP_HABITATION','CLNT_JOB_POSITION_TYPE','APP_EDUCATION','APP_POSITION_TYPE','APP_EMP_TYPE','PACK']

for cols in cat_cols_to_fill:
  mode_vals =  df[cols].mode()[0]
  df[cols].fillna(mode_vals, inplace=True)

/tmp/ipykernel_34/1721861242.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[cols].fillna(mode_vals, inplace=True)


In [20]:
#filled numerical columns with the help of 'median'
for cols in df.columns:
  if df[cols].dtype in ['int64']:
    median_vals =  df[cols].median()
    df[cols].fillna(median_vals, inplace=True)

/tmp/ipykernel_34/2624436037.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[cols].fillna(median_vals, inplace=True)
/tmp/ipykernel_34/2624436037.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

In [21]:
#filled numerical columns with the help of 'mean'
for cols in df.columns:
  if df[cols].dtype in ['float64']:
    mean_vals =  df[cols].mean()
    df[cols].fillna(mean_vals, inplace=True)

/tmp/ipykernel_34/3406880752.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[cols].fillna(mean_vals, inplace=True)


In [22]:
df.isnull().sum().max()

0

# Feature Engineering

In [24]:
import numpy as np
num_cols = df.select_dtypes(include=[np.number]).columns

num_cols = num_cols.drop(['TARGET'])